In [1]:
# default_exp viseme_tabular.data

# Viseme tabular data

> Tabular data handling functions.

TODO: create some test/data/capture_session ... so we can test end-to-end quickly

In [2]:
#export
from expoco.core import *
from expoco.camera_capture import *
from pathlib import Path
import numpy as np
import pandas as pd
import json, cv2

import mediapipe as mp
mp_face_mesh = mp.solutions.face_mesh

In [3]:
#export
def viseme_tabular_dataset_from_capture_sessions(
        input_path='data/capture_sessions', 
        glob_pattern='*'):
    "Create a viseme tabular dataset from capture session images"
    input_path, dataset_id, data = Path(input_path), now(), []
    output_path = input_path.parent/f'viseme_tabular_dataset_{dataset_id}'
    output_path.mkdir()
    face_mesh = mp_face_mesh.FaceMesh(max_num_faces=1)
    landmark_ids = sorted(FaceLandmarks.pointer + FaceLandmarks.mouth)
    metadata = dict(input_path=path_to_str(input_path), output_path=path_to_str(output_path), 
                    glob_pattern=glob_pattern, session_metadata=[], start_date=now(), 
                    landmark_ids=landmark_ids,
                    column_names=landmark_ids_to_col_names(FaceLandmarks.pointer + FaceLandmarks.mouth))
    data = []
    for session_path in sorted(input_path.glob(glob_pattern)):
        with open(session_path/'metadata.json') as f:
            session_metadata = json.load(f)
        metadata['session_metadata'].append(session_metadata)
        for capture_count in range(1, session_metadata['count']+1):
            image = cv2.imread(f'{session_path}/{capture_count}.png')
            results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
            row = []
            for landmark_id in landmark_ids:
                landmark = results.multi_face_landmarks[0].landmark[landmark_id]
                for coord in ['x','y']:
                    row.append(getattr(landmark, coord))
            data.append(row)
    metadata['end_date'] = now()
    with open(output_path/'metadata.json', 'w') as f: json.dump(metadata, f, indent=2)
    np.save(output_path/'data.npy', np.array(data), allow_pickle=False)
    return output_path

In [4]:
_viseme_tabular_dataset_path = viseme_tabular_dataset_from_capture_sessions('test/data/capture_sessions')
_viseme_tabular_dataset_path

WindowsPath('test/data/viseme_tabular_dataset_20211203_201834')

In [5]:
#export
def read_viseme_tabular_dataset(dataset_path, to_numpy=True):
    "Return metadata and data (as a numpy array by default, pandas dataframe if `to_numpy=False`)"
    dataset_path = Path(dataset_path)
    data = np.load(dataset_path/'data.npy')
    with open(dataset_path/'metadata.json') as f:
        dataset_metadata = json.load(f)
    viseme_class = []
    for session_metadata in dataset_metadata['session_metadata']:
        viseme_class.extend([get_viseme_class(session_metadata)]*session_metadata['count'])
    if to_numpy: 
        return dataset_metadata, data, viseme_class
    df = pd.DataFrame(data, columns=dataset_metadata['column_names'])
    df = pd.concat([df, pd.DataFrame(dict(viseme_class=viseme_class))], axis=1)
    return dataset_metadata, df

In [6]:
_viseme_tabular_dataset = read_viseme_tabular_dataset(_viseme_tabular_dataset_path)
print(_viseme_tabular_dataset[0]['column_names'][:3],
      _viseme_tabular_dataset[1].shape, _viseme_tabular_dataset[1].dtype,
      len(_viseme_tabular_dataset[2]), 
      _viseme_tabular_dataset[2][:3])

['0x', '0y', '1x'] (15, 214) float64 15 ['NO_EXPRESSION', 'NO_EXPRESSION', 'NO_EXPRESSION']


In [7]:
_viseme_tabular_dataset = read_viseme_tabular_dataset(_viseme_tabular_dataset_path, False)
_viseme_tabular_dataset[1]

,0x,0y,1x,1y,2x,2y,5x,5y,11x,11y,...,409y,410x,410y,415x,415y,424x,424y,438x,438y,viseme_class
0,0.567667,0.638635,0.570614,0.583237,0.568014,0.598804,0.570886,0.542354,0.567370,0.646737,...,0.651294,0.616236,0.639687,0.595893,0.653160,0.609032,0.687709,0.593685,0.579238,NO_EXPRESSION
1,0.566299,0.641875,0.569753,0.583833,0.567012,0.599397,0.570418,0.543183,0.565939,0.650085,...,0.655350,0.614223,0.642750,0.594308,0.656557,0.606654,0.688945,0.592635,0.579636,NO_EXPRESSION
2,0.566291,0.639309,0.568263,0.583081,0.566249,0.598477,0.569099,0.541882,0.566056,0.647434,...,0.652358,0.614565,0.640333,0.594835,0.653605,0.606685,0.687566,0.591464,0.578804,NO_EXPRESSION
3,0.564521,0.639694,0.566761,0.582529,0.564776,0.598240,0.567678,0.541042,0.564226,0.647984,...,0.652902,0.613718,0.640893,0.593790,0.654312,0.605515,0.689389,0.590404,0.578430,NO_EXPRESSION
4,0.565974,0.639391,0.568494,0.582082,0.566382,0.597709,0.569246,0.541081,0.565689,0.647591,...,0.651923,0.615296,0.639715,0.595522,0.653221,0.607298,0.687210,0.592084,0.577885,NO_EXPRESSION
5,0.541801,0.604615,0.540703,0.546617,0.541541,0.566375,0.541662,0.505464,0.541849,0.612958,...,0.628141,0.598688,0.616598,0.577566,0.630405,0.595152,0.680397,0.566866,0.545349,AH
6,0.546405,0.606775,0.545509,0.548134,0.546069,0.567782,0.545881,0.506726,0.546437,0.615201,...,0.630330,0.602037,0.618020,0.581306,0.632367,0.598535,0.681323,0.571330,0.546669,AH
7,0.544614,0.604672,0.543695,0.548541,0.544459,0.567843,0.544714,0.506438,0.544609,0.612955,...,0.628870,0.601318,0.617123,0.580388,0.630822,0.596879,0.681304,0.570153,0.547291,AH
8,0.546058,0.603855,0.545534,0.547314,0.545973,0.566985,0.546512,0.505842,0.546009,0.612230,...,0.629121,0.602658,0.617210,0.581632,0.631165,0.598036,0.681259,0.571963,0.546287,AH
9,0.545456,0.605273,0.544225,0.547993,0.544980,0.567712,0.545011,0.506245,0.545476,0.613702,...,0.630034,0.602087,0.618295,0.581269,0.631922,0.597730,0.682393,0.570512,0.546999,AH


In [8]:
#export
def get_image_path(dataset_path, row_idx):
    dataset_path = Path(dataset_path)
    if dataset_path.name.startswith('processed'):
        # TODO: read processed metadata and find dataset path - don't just assume its the parent
        dataset_path = dataset_path.parent
    with open(dataset_path/'metadata.json') as f:
        dataset_metadata = json.load(f)
    idx = row_idx
    for session_metadata in dataset_metadata['session_metadata']:
        if idx < session_metadata['count']:
            return Path(session_metadata['path'])/f'{idx+1}.jpeg'
        idx -= session_metadata['count']

In [9]:
assert Path('test/data/capture_sessions/20211203_171210/1.jpeg') == get_image_path(_viseme_tabular_dataset_path, 5)

In [10]:
#exporti
def select_columns(dataset_metadata, data, column_names):
    return data[..., [c in column_names for c in dataset_metadata['column_names']]]

In [11]:
_dataset_metadata, _data, _viseme_class = read_viseme_tabular_dataset(_viseme_tabular_dataset_path, True)
assert (15, 214) == _data.shape
_cont_names = landmark_ids_to_col_names(FaceLandmarks.pointer + FaceLandmarks.mouth, [FaceLandmarks.tip_of_nose])
assert len(_cont_names) == 212
_data = select_columns(_dataset_metadata, _data, _cont_names + ['expression_id'])
assert (15, 212) == _data.shape

In [12]:
#export
def make_landmarks_relative(data, to_landmarks, column_count=None):
    "Make all landmarks in `data` relative `to_landmarks` in-place"
    coord_count = to_landmarks.shape[-1]
    assert coord_count == 2, f'to_landmarks must have 2 "coord_count". to_arr.shape={coord_count}'
    for i in range(coord_count):
        data[:, i:column_count:coord_count] -= to_landmarks[..., i][..., None]
    return data

Columns in `arr` must be ordered as pairs of landmark IDs. e.g. `'5x', '5y', '2x', '2y', '218x', '218y' ...`

- `arr` 2d np array (rows, columns)
- `to_arr` 2d np array (rows, exactly 2 columns; `x` then `y`)
    - `to_arr.shape[0]` must be the same as `arr.shape[0]` - i.e. they must have the same number of rows
    
TODO: we could allow `to_arr.shape[1]` to be 1, 2 or 3 but ... leave it as is for now - might be hard to debug data issues if we accidentaly pass 1 column when we should be passing 2.

In [13]:
_arr = np.array([[.1,.2,.3,.4,0],
                [.2,.2,.3,.4,1],
                [.2,.6,.3,.6,2]])
_to_arr = np.array([[.0,.0],
                   [.2,.4],
                   [.8,.9]])
make_landmarks_relative(_arr, _to_arr, _arr.shape[-1]-1)

array([[ 0.1,  0.2,  0.3,  0.4,  0. ],
       [ 0. , -0.2,  0.1,  0. ,  1. ],
       [-0.6, -0.3, -0.5, -0.3,  2. ]])

In [14]:
_arr = np.array([[.1,.2,.3,.4,0]])
_to_arr = np.array([[-.3,.8]])
make_landmarks_relative(_arr, _to_arr)

array([[ 0.4, -0.6,  0.6, -0.4,  0.3]])

In [15]:
_dataset_metadata, _data, _viseme_class = read_viseme_tabular_dataset(_viseme_tabular_dataset_path, True)
_cont_names = landmark_ids_to_col_names(FaceLandmarks.pointer + FaceLandmarks.mouth, [FaceLandmarks.tip_of_nose])
_cont_data = select_columns(_dataset_metadata, _data, _cont_names + ['expression_id'])
_tip_of_nose_columns = landmark_ids_to_col_names([FaceLandmarks.tip_of_nose])
_tip_of_nose_data = select_columns(_dataset_metadata, _data, _tip_of_nose_columns)
_relative_data = make_landmarks_relative(_cont_data, _tip_of_nose_data, len(_cont_names))
assert (15, 212) == _relative_data.shape

In [16]:
pd.concat([pd.DataFrame(_relative_data, columns=_cont_names), pd.DataFrame(dict(viseme_class=_viseme_class))], axis=1)

,0x,0y,2x,2y,5x,5y,11x,11y,12x,12y,...,409y,410x,410y,415x,415y,424x,424y,438x,438y,viseme_class
0,-0.002947,0.055398,-0.002600,0.015568,0.000272,-0.040883,-0.003244,0.063500,-0.003950,0.069770,...,0.068057,0.045622,0.056450,0.025279,0.069923,0.038418,0.104472,0.023071,-0.003999,NO_EXPRESSION
1,-0.003453,0.058043,-0.002741,0.015564,0.000665,-0.040650,-0.003814,0.066253,-0.004501,0.072241,...,0.071517,0.044471,0.058917,0.024556,0.072724,0.036902,0.105112,0.022882,-0.004197,NO_EXPRESSION
2,-0.001972,0.056228,-0.002015,0.015396,0.000836,-0.041199,-0.002207,0.064353,-0.002782,0.070304,...,0.069278,0.046301,0.057252,0.026572,0.070524,0.038421,0.104485,0.023201,-0.004277,NO_EXPRESSION
3,-0.002241,0.057165,-0.001986,0.015710,0.000917,-0.041487,-0.002535,0.065454,-0.003153,0.071579,...,0.070373,0.046957,0.058364,0.027029,0.071783,0.038754,0.106859,0.023643,-0.004099,NO_EXPRESSION
4,-0.002519,0.057309,-0.002111,0.015627,0.000752,-0.041001,-0.002804,0.065509,-0.003403,0.071435,...,0.069841,0.046803,0.057633,0.027028,0.071139,0.038804,0.105128,0.023591,-0.004197,NO_EXPRESSION
5,0.001098,0.057998,0.000838,0.019758,0.000959,-0.041153,0.001146,0.066341,0.001280,0.073704,...,0.081524,0.057984,0.069981,0.036863,0.083789,0.054449,0.133780,0.026163,-0.001267,AH
6,0.000896,0.058641,0.000560,0.019648,0.000373,-0.041408,0.000928,0.067066,0.001032,0.074423,...,0.082196,0.056529,0.069886,0.035798,0.084233,0.053027,0.133188,0.025822,-0.001465,AH
7,0.000919,0.056131,0.000764,0.019302,0.001019,-0.042103,0.000914,0.064414,0.001009,0.071808,...,0.080329,0.057623,0.068582,0.036693,0.082281,0.053184,0.132764,0.026458,-0.001250,AH
8,0.000524,0.056540,0.000439,0.019671,0.000979,-0.041472,0.000475,0.064916,0.000557,0.072557,...,0.081807,0.057124,0.069896,0.036099,0.083851,0.052502,0.133945,0.026429,-0.001027,AH
9,0.001231,0.057280,0.000755,0.019719,0.000786,-0.041749,0.001251,0.065708,0.001389,0.073237,...,0.082040,0.057862,0.070302,0.037044,0.083928,0.053505,0.134400,0.026287,-0.000995,AH


In [17]:
#export
def change_viseme_class(viseme_class, from_class, to_class):
    "Change values in viseme_class from one class to another"
    return [to_class if c==from_class else c for c in viseme_class]

In [18]:
#export
def calculate_stats(data):
    "Return the mean and standard deviation over columns of `data`"
    return dict(mean=data.mean(axis=0), std=data.std(axis=0))

In [19]:
_dataset_metadata, _data, _viseme_class = read_viseme_tabular_dataset(_viseme_tabular_dataset_path, True)
_data = select_columns(_dataset_metadata, _data, _cont_names[:2])
_stats = calculate_stats(_data)
assert (2,) == _stats['mean'].shape
assert (2,) == _stats['std'].shape

In [20]:
#export
def normalize(data, mean, std):
    column_count = len(mean) # use len rather than shape[0] so that mean stats can be lists or np arrays
    data[..., :column_count] -= mean
    data[..., :column_count] /= std
    return data

In [21]:
_dataset_metadata, _data, _viseme_class = read_viseme_tabular_dataset(_viseme_tabular_dataset_path, True)
_data = select_columns(_dataset_metadata, _data, _cont_names[:2])
_stats = calculate_stats(_data)
_normalized_data = normalize(_data, **_stats)
assert np.allclose(_normalized_data.mean(axis=0), 0)
assert np.allclose(_normalized_data.std(axis=0), 1)

# Combine data processing steps

In [22]:
#export
def inference_data_from_landmarks(landmarks, landmark_ids, relative_landmark_id=None, coords=['x', 'y'], stats=None):
    landmark_ids, coords, data, to_landmarks_data = sorted(landmark_ids), sorted(coords), [], []
    for landmark_id in landmark_ids:
        for coord in coords:
            if landmark_id == relative_landmark_id:
                to_landmarks_data.append(getattr(landmarks[landmark_id], coord))
            else:
                data.append(getattr(landmarks[landmark_id], coord))
    data, to_landmarks_data = np.array([data], dtype=float), np.array([to_landmarks_data], dtype=float)
    if relative_landmark_id is not None:
        make_landmarks_relative(data, to_landmarks_data)
    if stats is not None:
        normalize(data, **stats)
    return data

In [23]:
from collections import namedtuple
_Landmark = namedtuple('_Landmark', ['x','y','z'])
_landmarks=[]
for i in range(FaceLandmarks.count):
    _landmarks.append(_Landmark(i+.1, i+.2, i+.3))
assert [[-1., -1.,  1.,  1.]] == inference_data_from_landmarks(_landmarks, [0,2,1], 1).round().tolist()

In [24]:
_stats = dict(mean=[0.1,0.2,0.3,0.4], std=[0.1,0.2,0.3,0.4])
_inference_data = inference_data_from_landmarks(
        landmarks=_landmarks, landmark_ids=[2,0,1], relative_landmark_id=1, stats=_stats)
assert [[-11.0,-6.0,2.3,1.5]] == np.round(_inference_data, 1).tolist()

In [25]:
#export
def processed_dataset_from_viseme_tabular_dataset(
        input_path, relative_landmark_id=None, change_y_from=None, change_y_to=None):
    "Create a processed dataset (ready for ML) from a viseme tabular dataset"
    input_path = Path(input_path)
    input_metadata, input_data, input_y = read_viseme_tabular_dataset(input_path)
    output_path = Path(input_metadata['output_path'])/f'processed_{now()}'
    output_path.mkdir()
    column_names = landmark_ids_to_col_names(input_metadata['landmark_ids'], relative_landmark_id)
    metadata = dict(input_path=path_to_str(input_path), 
                    input_metadata=input_metadata, 
                    output_path=path_to_str(output_path),
                    relative_landmark_id=relative_landmark_id, 
                    change_y_from=change_y_from,
                    change_y_to=change_y_to,
                    column_names=column_names,
                    start_date=now())
    data = select_columns(input_metadata, input_data, column_names)
    if change_y_from is not None:
        assert change_y_to is not None
        input_y = change_viseme_class(input_y, change_y_from, change_y_to)
    metadata['viseme_class'] = input_y
    if relative_landmark_id is not None:
        relative_names = landmark_ids_to_col_names([relative_landmark_id])
        metadata['relative_names']=relative_names
        to_landmarks = select_columns(input_metadata, input_data, relative_names)
        make_landmarks_relative(data, to_landmarks)
    stats = calculate_stats(data)
    normalize(data, **stats)
    metadata['end_date'] = now()
    with open(output_path/'metadata.json', 'w') as f: json.dump(metadata, f, indent=2)
    np.save(output_path/'data.npy', data, allow_pickle=False)
    np.savez(output_path/'stats.npz', **stats)
    return output_path

In [26]:
_processed_dataset_path = processed_dataset_from_viseme_tabular_dataset(
        input_path=_viseme_tabular_dataset_path, 
        relative_landmark_id=FaceLandmarks.tip_of_nose,
        change_y_from='RANDOM_TALK', change_y_to='NO_EXPRESSION')
_processed_dataset_path

WindowsPath('test/data/viseme_tabular_dataset_20211203_201834/processed_20211203_201835')

In [27]:
#export
def read_processed_dataset(dataset_path, to_numpy=True):
    "Return metadata, data and stats (as a numpy array by default, pandas dataframe if `to_numpy=False`)"
    dataset_path = Path(dataset_path)
    data, stats = np.load(dataset_path/'data.npy'), np.load(dataset_path/'stats.npz')
    with open(dataset_path/'metadata.json') as f:
        dataset_metadata = json.load(f)
    viseme_class = dataset_metadata['viseme_class']
    if to_numpy: 
        return dataset_metadata, data, viseme_class, stats
    column_names = dataset_metadata['column_names']
    stats = dict(mean=pd.DataFrame(stats['mean'][None, ...], columns=column_names),
                 std=pd.DataFrame(stats['std'][None, ...], columns=column_names))
    df = pd.DataFrame(data, columns=dataset_metadata['column_names'])
    df = pd.concat([df, pd.DataFrame(dict(viseme_class=viseme_class))], axis=1)
    return dataset_metadata, df, stats

In [28]:
_metadata, _data, _viseme_class, _stats = read_processed_dataset(_processed_dataset_path, True)

In [29]:
_metadata, _data, _stats = read_processed_dataset(_processed_dataset_path, False)
_data

,0x,0y,2x,2y,5x,5y,11x,11y,12x,12y,...,409y,410x,410y,415x,415y,424x,424y,438x,438y,viseme_class
0,-1.172053,-1.805594,-1.176585,-0.972314,-1.608066,1.462928,-1.150270,-1.857522,-1.170150,-1.891132,...,-1.667442,-0.869296,-1.471985,-0.794176,-1.561347,-0.969043,-1.195726,-1.258347,-0.659306,NO_EXPRESSION
1,-1.495341,0.762868,-1.283923,-0.974134,-0.136760,2.056744,-1.485213,0.489638,-1.443539,-0.258217,...,-0.977223,-1.089162,-0.978219,-0.934948,-1.025513,-1.203629,-1.141279,-1.409563,-0.803492,NO_EXPRESSION
2,-0.550119,-0.999342,-0.732147,-1.071276,0.502172,0.661125,-0.541080,-1.130241,-0.590839,-1.538788,...,-1.424024,-0.739518,-1.311557,-0.542780,-1.446482,-0.968527,-1.194555,-1.154418,-0.862508,NO_EXPRESSION
3,-0.721480,-0.089928,-0.709955,-0.890005,0.805704,-0.071298,-0.733612,-0.190991,-0.774557,-0.695762,...,-1.205600,-0.614306,-1.088985,-0.453802,-1.205660,-0.917117,-0.992708,-0.800593,-0.732532,NO_EXPRESSION
4,-0.899458,0.050632,-0.805428,-0.937873,0.189280,1.162839,-0.891890,-0.143932,-0.898860,-0.791204,...,-1.311637,-0.643736,-1.235155,-0.453976,-1.328733,-0.909316,-1.139895,-0.842394,-0.804145,NO_EXPRESSION
5,1.408703,0.719681,1.435508,1.441754,0.962373,0.778676,1.429180,0.565105,1.424771,0.708257,...,1.018988,1.492052,1.236707,1.459187,1.090800,1.511137,1.296603,1.216804,1.338497,AH
6,1.279859,1.343691,1.224454,1.378194,-1.230545,0.129872,1.301374,1.183530,1.301503,1.183734,...,1.152956,1.214019,1.217699,1.251932,1.175737,1.291074,1.246247,0.943622,1.193832,AH
7,1.294120,-1.093937,1.379574,1.179342,1.185231,-1.636882,1.293213,-1.077896,1.290412,-0.544662,...,0.780541,1.423082,0.956610,1.426085,0.802436,1.315392,1.210128,1.453196,1.351181,AH
8,1.042327,-0.696166,1.132650,1.391380,1.035470,-0.033124,1.035395,-0.650147,1.066265,-0.049687,...,1.075360,1.327721,1.219584,1.310520,1.102657,1.209971,1.310572,1.430005,1.514195,AH
9,1.493660,0.021744,1.372599,1.419022,0.315641,-0.736008,1.491070,0.025451,1.478774,0.400108,...,1.121934,1.468690,1.300950,1.494377,1.117524,1.365031,1.349271,1.316247,1.537732,AH


In [30]:
import shutil
shutil.rmtree(_viseme_tabular_dataset_path)

In [31]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted 01a_camera_capture.ipynb.
Converted 10a_viseme_tabular_identify_landmarks.ipynb.
Converted 10b_viseme_tabular_data.ipynb.
Converted 10d_viseme_tabular_model.ipynb.
Converted 10e_viseme_tabular_train_model.ipynb.
Converted 10f_viseme_tabular_test_model.ipynb.
Converted 11b_viseme_image_data.ipynb.
Converted 11d_viseme_image_model.ipynb.
Converted 11e_viseme_image_train_model.ipynb.
Converted 11f_viseme_image_test_model.ipynb.
Converted 20a_gui_capture_command.ipynb.
Converted 20a_gui_main.ipynb.
Converted 70_cli.ipynb.
Converted index.ipynb.
Converted project_lifecycle.ipynb.
